# Лабораторная работа №4

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
import os
os.chdir("C:/Users/вячеслав/Desktop/Lab_01/Lab_01/")

In [3]:
df = pd.read_csv('Winequality-red.csv', sep=';')

In [4]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
# смотрим на размерность матрицы данных
df.shape

(1599, 12)

In [6]:
# смотрим сбалансированы ли классы, они не сбалансированы 
df['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [7]:
df['quality'] = df['quality'].replace(to_replace=3, value=4).replace(8, 7)

In [8]:
df['quality'].value_counts()

5    681
6    638
7    217
4     63
Name: quality, dtype: int64

In [9]:
#  разделяю предикоторы и отклики
y = df['quality']
X = df.drop(columns=['quality'])

In [10]:
#  разделяю на обучающую и тестовую выборку
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 1234) 

In [11]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [12]:
grid_param = {  
    'n_estimators': [100, 300, 400, 500, 800],
    'max_features': ['sqrt', 5, 7, 9],
    'max_depth': [3, 4, 5, 6, 7],
    'reg_alpha': [0, 1],
    'reg_lambda': [0, 1]
}

In [13]:
grid_search_res = GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, seed=1234, 
                               
       objective='multy:softprob', num_class = 4,                  
       colsample_bylevel=1, colsample_bytree=0.8, subsample=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, 
       min_child_weight=1, missing=None, nthread=-1,
       reg_alpha=0, reg_lambda=1, 
       scale_pos_weight=1, silent=True), n_jobs=-1,
       param_grid=grid_param,
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [14]:
grid_search_res.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:14:10] WARNING: ..\src\learner.cc:541: 
Parameters: { max_features, scale_pos_weight, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:14:10] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5, error_score='raise',
             estimator=XGBClassifier(base_score=0.5, booster=None,
                                     colsample_bylevel=1, colsample_bynode=None,
                                     colsample_bytree=0.8, gamma=0, gpu_id=None,
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=None, min_child_weight=1,
                                     missing=None, monotone_constraints=None,
                                     n_estimators=100,...
                                     reg_lambda=1, scale_pos_weight=1,
                                     seed=1234, silent=True, subsample=0.8,
                                     tree_method=None, use_label_encoder=True, ...),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7

In [15]:
print("Best params on dev set:")
print(grid_search_res.best_params_)

Best params on dev set:
{'max_depth': 6, 'max_features': 'sqrt', 'n_estimators': 100, 'reg_alpha': 1, 'reg_lambda': 1}


In [16]:
# строю предсказание модели
y_pred_train = grid_search_res.predict(X_train)
y_pred_test = grid_search_res.predict(X_test)

In [17]:
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,4,5,6,7
4,0,12,2,2
5,2,181,47,1
6,1,48,138,30
7,0,2,24,38


In [18]:
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,4,5,6,7
4,45,1,1,0
5,0,444,5,1
6,0,4,417,0
7,0,2,2,149


In [19]:
# строим предсказание модели
y_pred_train2 = grid_search_res.predict_proba(X_train)
y_pred_test2 = grid_search_res.predict_proba(X_test)
print(y_pred_train2)
print(y_pred_test2)

[[0.00181549 0.02515561 0.04660438 0.9264245 ]
 [0.00521301 0.25389254 0.7372816  0.00361287]
 [0.01008777 0.98012173 0.00871464 0.00107581]
 ...
 [0.01845155 0.2384336  0.73899657 0.00411826]
 [0.01043152 0.7569116  0.17312169 0.05953518]
 [0.00302374 0.7336641  0.18531637 0.07799582]]
[[0.13899823 0.751474   0.08838799 0.02113977]
 [0.0135957  0.62012434 0.34482488 0.02145505]
 [0.0104693  0.12199611 0.6895515  0.17798312]
 ...
 [0.04738577 0.11394127 0.5039713  0.33470172]
 [0.00236957 0.02068698 0.35920888 0.6177346 ]
 [0.01028693 0.68336296 0.3008252  0.00552494]]


In [20]:
y_pred_train2 = grid_search_res.predict(X_train)
y_pred_test2 = grid_search_res.predict(X_test)
print(metrics.classification_report(y_pred_test2, y_test))

              precision    recall  f1-score   support

           4       0.00      0.00      0.00         3
           5       0.78      0.74      0.76       243
           6       0.64      0.65      0.64       211
           7       0.59      0.54      0.56        71

    accuracy                           0.68       528
   macro avg       0.50      0.48      0.49       528
weighted avg       0.69      0.68      0.68       528



In [21]:
fi = pd.DataFrame({'features': X_train.columns, 'importance': grid_search_res.best_estimator_.feature_importances_})
fi.sort_values('importance', ascending=False).head(10)

,features,importance
10,alcohol,0.198420
9,sulphates,0.116409
1,volatile acidity,0.104514
6,total sulfur dioxide,0.090997
0,fixed acidity,0.075658
8,pH,0.074629
4,chlorides,0.071708
5,free sulfur dioxide,0.069622
7,density,0.068088
3,residual sugar,0.066533


In [22]:
# оцениваем долю наблюдений в тестовой выборке, для которых есть класс,вероятность принадлежать которому больше 0.8
# до калибровки
y_pred_train2 = grid_search_res.predict_proba(X_train)
y_pred_test2 = grid_search_res.predict_proba(X_test)
k = 0
for i in y_pred_test2:
    for j in i:
        if (j>0.8):
            k += 1
dolya_do = k/len(y_pred_test2)*100

In [23]:
dolya_do

34.46969696969697

# Делаем калибровку

In [24]:
from sklearn.calibration import CalibratedClassifierCV

In [25]:
model_sigmoid = CalibratedClassifierCV(grid_search_res, cv=2, method='sigmoid')

In [26]:
model_sigmoid.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:31:37] WARNING: ..\src\learner.cc:541: 
Parameters: { max_features, scale_pos_weight, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:31:37] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:49:33] WARNING: ..\src\learner.cc:541: 
Parameters: { max_features, scale_pos_weight, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:49:33] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


CalibratedClassifierCV(base_estimator=GridSearchCV(cv=5, error_score='raise',
                                                   estimator=XGBClassifier(base_score=0.5,
                                                                           booster=None,
                                                                           colsample_bylevel=1,
                                                                           colsample_bynode=None,
                                                                           colsample_bytree=0.8,
                                                                           gamma=0,
                                                                           gpu_id=None,
                                                                           importance_type='gain',
                                                                           interaction_constraints=None,
                                                                           

In [27]:
y_pred_train3 = model_sigmoid.predict(X_train)
y_pred_test3 = model_sigmoid.predict(X_test)
print(metrics.classification_report(y_pred_test3, y_test))

              precision    recall  f1-score   support

           4       0.00      0.00      0.00         1
           5       0.79      0.72      0.75       254
           6       0.58      0.60      0.59       210
           7       0.44      0.44      0.44        63

    accuracy                           0.64       528
   macro avg       0.45      0.44      0.45       528
weighted avg       0.66      0.64      0.65       528



In [28]:
y_pred_train3 = model_sigmoid.predict_proba(X_train)
y_pred_test3 = model_sigmoid.predict_proba(X_test)
print(y_pred_train3)
print(y_pred_test3)

[[0.02879019 0.14710323 0.25617019 0.5679364 ]
 [0.03309871 0.45028668 0.48640498 0.03020964]
 [0.03459775 0.80304142 0.13393549 0.02842534]
 ...
 [0.03457143 0.49668635 0.43879336 0.02994886]
 [0.03409198 0.721881   0.19128991 0.05273711]
 [0.03426702 0.3744984  0.50240026 0.08883433]]
[[0.06775343 0.73908695 0.15905379 0.03410583]
 [0.03321167 0.64391018 0.28988786 0.03299029]
 [0.03689712 0.21973941 0.63877501 0.10458845]
 ...
 [0.033932   0.13221839 0.42667565 0.40717396]
 [0.02804463 0.10740955 0.33865668 0.52588915]
 [0.03369852 0.59150191 0.3447668  0.03003276]]
